In [1]:
pip install mne

Note: you may need to restart the kernel to use updated packages.


In [2]:
import mne
import os
import csv
import pickle
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [3]:
def read_eeg_signal_from_file(filename):
    x = pickle._Unpickler(open(filename, 'rb'))
    x.encoding = 'latin1'
    p = x.load()
    return p

In [4]:
files = []
for n in range(1, 21): 
    s = ''
    if n < 10:
        s += '0'
    s += str(n)
    files.append(s)


In [5]:
labels = []
data = []

for i in files: 
  filename = "Data/s" + i + ".dat"
  trial = read_eeg_signal_from_file(filename)
  labels.append(trial['labels'])
  data.append(trial['data'])

In [6]:
labels = np.array(labels)
data = np.array(data)
print("Labels: ", labels.shape) 
print("Data: ", data.shape) 

Labels:  (20, 40, 4)
Data:  (20, 40, 40, 8064)


In [7]:
labels = np.array(labels)
labels = labels.flatten()
labels = labels.reshape(800, 4)
data = np.array(data)
data = data.flatten()
data = data.reshape(800, 40, 8064)
print("Labels: ", labels.shape) 
print("Data: ", data.shape) 

Labels:  (800, 4)
Data:  (800, 40, 8064)


In [8]:
eeg_data = []
for i in range(len(data)):
    for j in range(40):
        if(j in [1 , 2 , 3 , 4 , 7 , 11 , 13 , 17 , 19 , 20 , 21 , 25 , 29 , 31]):
            eeg_data.append(data[i][j])
eeg_data = np.array(eeg_data)
eeg_data = eeg_data.reshape(len(data) , 14 , len(data[0 , 0]))
eeg_data.shape

(800, 14, 8064)

In [9]:
labels_enc = []
for i in range(len(labels)):
    cmp = (float)(5)
    put = [(float)(0) , (float)(0)]
    if(labels[i][0] > cmp):
        put[0] = 1
    if(labels[i][1] > cmp):
        put[1] = 1
    labels_enc.append(put)

In [10]:
def get_raw_eeg(DATA):
    ch_names = np.array(["AF3", "F3", "F7", "FC5", "T7", "P7", "O1", "AF4", "F4", "F8", "FC6", "T8", "P8", "O2"])
    ch_types = ['eeg'] * 14
    sampling_freq=128
    info = mne.create_info(ch_names.tolist(), ch_types=ch_types, sfreq=sampling_freq)
    info.set_montage('standard_1020')
    raw_data = mne.io.RawArray(DATA, info)
    raw_data.set_eeg_reference()
    raw_data.filter(l_freq=1,h_freq=30)
    epochs = mne.make_fixed_length_epochs(raw_data,duration=2,overlap=0)
    return epochs.get_data()

In [11]:
%%capture
data_epochs_array = []
for i in range(len(eeg_data)):
    data_epochs_array.append(get_raw_eeg(eeg_data[i]))

In [12]:
data_epochs_array[0].shape , data_epochs_array[21].shape #2nd -> no. of epochs , 3rd -> channels , 4th -> length of signal

((31, 14, 256), (31, 14, 256))

In [13]:
def get_quadrant(l):
    ret = np.zeros(4 , dtype = float)
    idx = 0
    if(l[0] == 1):
        idx += 1
    if(l[1] == 1):
        idx += 2
    ret[idx] = (float)(1)
    return ret

In [14]:
data_epochs_array = np.array(data_epochs_array)
training_data = []
#trial x epoch x channel x data_point
for i in range(len(data_epochs_array)):
    for j in range(31):
        for k in range(14):
            for w in range(256):
                training_data.append([i , j , k , w , data_epochs_array[i][j][k][w]])
        
data_epochs_array.shape

(800, 31, 14, 256)

In [15]:
training_data = np.array(training_data)
training_data.shape

(88883200, 5)

In [16]:
label_epochs_array = []
for i in range(len(eeg_data)):
    put = []
    for j in range(31):
        p1 = []
        for k in range(14):
            p1.append(get_quadrant(labels_enc[i]))
        put.append(p1)
    label_epochs_array.append(put)
label_epochs_array = np.array(label_epochs_array)
label_epochs_array.shape

(800, 31, 14, 4)

In [17]:
training_labels = []
for i in range(len(label_epochs_array)):
    for j in range(31):
        for k in range(14):
            for w in range(256):
                training_labels.append(label_epochs_array[i][j][k])
            
training_labels = np.array(training_labels)
training_labels.shape

(88883200, 4)

In [19]:
from sklearn.model_selection import train_test_split
def train_val_test(x , y):
    x_train, x_b, y_train, y_b = train_test_split(x,y , random_state=0, train_size = .6)
    x_val , x_test , y_val , y_test = train_test_split(x_b , y_b , random_state = 0 , train_size = 0.5)
    return x_train , x_val , x_test , y_train , y_val , y_test
    
    

In [20]:
x_train , x_val , x_test , y_train , y_val , y_test = train_val_test(training_data , training_labels)

In [21]:
x_train.shape , y_train.shape

((53329920, 5), (53329920, 4))

In [22]:
x_val.shape , y_val.shape

((17776640, 5), (17776640, 4))

In [23]:
x_test.shape , y_test.shape

((17776640, 5), (17776640, 4))

In [24]:
from numpy import asarray
from sklearn.preprocessing import MinMaxScaler

to_scale = asarray(x_train)
scaler = MinMaxScaler()
# transform data
x_1s_train = scaler.fit_transform(x_train)
x_1s_val = scaler.transform(x_val)
x_1s_test = scaler.transform(x_test)

In [25]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_s_train = scaler.fit_transform(x_1s_train)
x_s_val = scaler.transform(x_1s_val)
x_s_test = scaler.transform(x_1s_test)

In [26]:
x_s_train = x_s_train.reshape(53329920, 5 , 1)

In [27]:
x_s_val = x_s_val.reshape(17776640, 5 , 1)

In [28]:
x_s_test = x_s_test.reshape(17776640, 5 , 1)

In [29]:
from tensorflow.keras.layers import Input,Dense,concatenate,Flatten,LSTM,Conv1D,MaxPooling1D , Dropout , GRU
from tensorflow.keras.models import Model , Sequential

In [30]:
def block(input):
  conv1 = Conv1D(32, 2, strides=2,activation='relu',padding="same")(input)
  conv2 = Conv1D(32, 4, strides=2,activation='relu',padding="causal")(input)
  conv3 = Conv1D(32, 8, strides=2,activation='relu',padding="causal")(input)
  x = concatenate([conv1,conv2,conv3],axis=2)
  return x

In [31]:
input= Input(shape=(5 , 1))
block1=block(input)
block2=block(block1)
block3=block(block2)


2022-12-14 19:52:14.954046: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [32]:
gru_out1 = GRU(32,activation='tanh',return_sequences=True)(block3)
gru_out2 = GRU(32,activation='tanh',return_sequences=True)(gru_out1)
gru_out = concatenate([gru_out1,gru_out2],axis=2)
gru_out3 = GRU(32,activation='tanh',return_sequences=True)(gru_out)
gru_out = concatenate([gru_out1,gru_out2,gru_out3])
gru_out4 = GRU(32,activation='tanh')(gru_out)

In [33]:
predictions = Dense(1,activation='sigmoid')(gru_out4)
model = Model(inputs=input, outputs=predictions)

model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=['accuracy'])

In [34]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 5, 1)]       0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 3, 32)        96          ['input_1[0][0]']                
                                                                                                  
 conv1d_1 (Conv1D)              (None, 3, 32)        160         ['input_1[0][0]']                
                                                                                                  
 conv1d_2 (Conv1D)              (None, 3, 32)        288         ['input_1[0][0]']                
                                                                                              

In [41]:
model.fit(x_s_train , y_train[: , 0] , epochs = 1 , batch_size = 256 , validation_data=(x_s_val , y_val[: , 0]))

208320/208320 [==============================] - 2784s 13ms/step - loss: 0.2793 - accuracy: 0.8713 - val_loss: 0.2834 - val_accuracy: 0.8674


In [42]:
results = model.evaluate(x_s_test, y_test[: , 0], batch_size = 256)
print("test loss, test acc:", results)

69440/69440 [==============================] - 197s 3ms/step - loss: 0.2834 - accuracy: 0.8674
test loss, test acc: [0.283426433801651, 0.8674381375312805]


In [39]:
model.fit(x_s_train , y_train[: , 1] , epochs = 1 , batch_size = 256 , validation_data=(x_s_val , y_val[: , 1]))

208320/208320 [==============================] - 2248s 11ms/step - loss: 0.2531 - accuracy: 0.8895 - val_loss: 0.2103 - val_accuracy: 0.9045


In [40]:
results = model.evaluate(x_s_test, y_test[: , 1], batch_size = 256)
print("test loss, test acc:", results)

69440/69440 [==============================] - 216s 3ms/step - loss: 0.2102 - accuracy: 0.9046
test loss, test acc: [0.21015439927577972, 0.9046063423156738]


In [35]:
model.fit(x_s_train , y_train[: , 2] , epochs=2 , batch_size = 256 , validation_data=(x_s_val , y_val[: , 2]))

Epoch 1/2
208320/208320 [==============================] - 4333s 21ms/step - loss: 0.2268 - accuracy: 0.8980 - val_loss: 0.3411 - val_accuracy: 0.8664
Epoch 2/2
208320/208320 [==============================] - 2200s 11ms/step - loss: 0.1908 - accuracy: 0.9157 - val_loss: 0.1368 - val_accuracy: 0.9401


In [36]:
results = model.evaluate(x_s_test, y_test[: , 2], batch_size = 256)
print("test loss, test acc:", results)

69440/69440 [==============================] - 194s 3ms/step - loss: 0.1369 - accuracy: 0.9401
test loss, test acc: [0.13691455125808716, 0.9400691986083984]


In [37]:
model.fit(x_s_train , y_train[: , 3] , epochs = 2 , batch_size = 256 , validation_data=(x_s_val , y_val[: , 3]))

Epoch 1/2
208320/208320 [==============================] - 1938s 9ms/step - loss: 0.2305 - accuracy: 0.8950 - val_loss: 0.1951 - val_accuracy: 0.9089
Epoch 2/2
208320/208320 [==============================] - 2611s 13ms/step - loss: 0.2338 - accuracy: 0.8894 - val_loss: 0.2022 - val_accuracy: 0.8986


In [38]:
results = model.evaluate(x_s_test, y_test[: , 3], batch_size = 256)
print("test loss, test acc:", results)

69440/69440 [==============================] - 210s 3ms/step - loss: 0.2021 - accuracy: 0.8987
test loss, test acc: [0.20206302404403687, 0.8987424373626709]
